In [39]:
import pickle
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, auc, roc_auc_score
import numpy as np

# Load images from a pickle file
with open("II_test.pickle", "rb") as tr_images_file:
    test = pickle.load(tr_images_file)
with open("II_train.pickle", "rb") as tr_images_file:
    train = pickle.load(tr_images_file)
with open("II_val.pickle", "rb") as tr_images_file:
    new = pickle.load(tr_images_file)
with open("II_deg.pickle", "rb") as tr_images_file:
    deg = pickle.load(tr_images_file)


#train=np.concat(train, img_tmp)

so far so good...


In [40]:

def unpk(data):

    x=[]
    y=[]
    c=0
    tot=[]
    for image, labels in data:
        try:
            img=np.array(image).reshape((150,150,3))
            x.append(img)
            y.append(labels)
        except Exception as e:
            c=c+1

    print("exceptions")
    print(c)
    return x, y

x_train,y_train=unpk(train)
x_test,y_test=unpk(test)
x_new, y_new=unpk(new)
x_deg, y_deg=unpk(deg)
c2=0


eccezioni
16
eccezioni
19
eccezioni
16
eccezioni
4


In [41]:
new_train_x=x_train.copy()
new_train_y=y_train.copy()

new_x_test=[]
new_y_test=[]

for i in range(len(x_test)):
    if i <4000:
        new_x_test.append(x_test[i])
        new_y_test.append(y_test[i])
    else:
        new_train_x.append(x_test[i])
        new_train_y.append(y_test[i])
        
for i in range(60):
    new_train_x.append(new_train_x[i])
    new_train_y.append(new_train_y[i])
print(len(new_train_x))
print(len(new_x_test))

new_train_x=np.array(new_train_x)
new_test_x=np.array(new_x_test)
new_train_y=np.array(new_train_y)
new_test_y=np.array(new_y_test)
x_new=np.array(x_new)
x_deg=np.array(x_deg)
y_new=np.array(y_new)
y_deg=np.array(y_deg)

6000
4000


In [42]:
print(len(new_train_x))
print(len(new_test_x))
print(len(x_new))
print(len(x_deg))
print(type(new_train_x))
print(type(new_train_x))
print(type(x_new))
print(type(x_deg))

6000
4000
4259
2780
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>


In [43]:
from tensorflow.keras.utils import to_categorical

print(new_train_y)

cat_lab_train=to_categorical(new_train_y)
cat_lab_test=to_categorical(new_test_y)
cat_lab_new=to_categorical(y_new)
cat_lab_deg=to_categorical(y_deg)

[4 0 4 ... 3 0 1]


In [44]:
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.utils import to_categorical
import numpy as np
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Flatten, Dense


base_model=VGG16(weights='imagenet', include_top=False, input_shape=(150,150,3))


for layer in base_model.layers[:]:
        layer.trainable = False
        
model=Sequential()
model.add(base_model)
model.add(Flatten(name="fl"))
model.add(Dense(256, activation="relu", name="fc"))
model.add(Dense(5, activation="softmax"))

model.compile(loss="categorical_crossentropy", optimizer="sgd", metrics=["acc"])

model.fit(new_train_x, cat_lab_train, batch_size=50, epochs=10, validation_split=0)


model_e= Model(inputs=model.input, outputs=model.get_layer('fc').output)
#model_e= base_model


Epoch 1/10
120/120 [==============================] - 50s 409ms/step - loss: 2.1141 - acc: 0.8183
Epoch 2/10
120/120 [==============================] - 49s 412ms/step - loss: 0.1913 - acc: 0.9277
Epoch 3/10
120/120 [==============================] - 49s 411ms/step - loss: 0.1099 - acc: 0.9615
Epoch 4/10
120/120 [==============================] - 50s 413ms/step - loss: 0.0670 - acc: 0.9763
Epoch 5/10
120/120 [==============================] - 49s 412ms/step - loss: 0.0365 - acc: 0.9898
Epoch 6/10
120/120 [==============================] - 49s 412ms/step - loss: 0.0175 - acc: 0.9963
Epoch 7/10
120/120 [==============================] - 49s 412ms/step - loss: 0.0082 - acc: 0.9988
Epoch 8/10
120/120 [==============================] - 50s 413ms/step - loss: 0.0063 - acc: 0.9993
Epoch 9/10
120/120 [==============================] - 50s 415ms/step - loss: 0.0046 - acc: 0.9992
Epoch 10/10
120/120 [==============================] - 50s 415ms/step - loss: 0.0049 - acc: 0.9992


In [45]:
print(model.summary())
#print(len(model))

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 4, 4, 512)         14714688  
                                                                 
 fl (Flatten)                (None, 8192)              0         
                                                                 
 fc (Dense)                  (None, 256)               2097408   
                                                                 
 dense_6 (Dense)             (None, 5)                 1285      
                                                                 
Total params: 16813381 (64.14 MB)
Trainable params: 2098693 (8.01 MB)
Non-trainable params: 14714688 (56.13 MB)
_________________________________________________________________
None


In [46]:
print(new_test_y[0])
print(new_train_y[0])
print(y_new[0])
print(y_deg[0])

4
4
4
5


In [47]:
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, auc, roc_auc_score


pred=model.predict(new_test_x)

print(confusion_matrix(new_test_y, pred.argmax(1)))

# Print classification report
print("Classification Report:")
print(classification_report(new_test_y, pred.argmax(1)))

125/125 [==============================] - 34s 274ms/step
[[755   2   7   2   7]
 [  4 666   3  62   4]
 [  8   3 667   2 142]
 [  0  60   4 745   2]
 [  1   0 111   2 741]]
Classification Report:
              precision    recall  f1-score   support

           0       0.98      0.98      0.98       773
           1       0.91      0.90      0.91       739
           2       0.84      0.81      0.83       822
           3       0.92      0.92      0.92       811
           4       0.83      0.87      0.85       855

    accuracy                           0.89      4000
   macro avg       0.90      0.89      0.90      4000
weighted avg       0.89      0.89      0.89      4000



In [48]:

def embedding_creation(X, y):
    predictions=model.predict(X)
    #conv_embedding=base_model.predict(X)
    embedding=model_e.predict(X)

    emb_list=[]
    
    prediction_list=[]

    for i in range(len(X)):
        emb_list.append(embedding[i])
        #conv_emb_list.append(conv_embedding[i])
        prediction_list.append(predictions[i])

    emb_list=np.array(emb_list)
    #conv_emb_list=np.array(conv_emb_list)

    prediction_list=np.array(prediction_list)
    prediction_list=np.argmax(prediction_list, axis=1)
    

    return prediction_list, emb_list,  y
    
def save_embeddings(embedding, predicted_label, true_label, path):
    fp = h5py.File(path, "w")
    fp.create_dataset("embedding", data=embedding, compression="gzip")
    fp.create_dataset("predicted_label", data=predicted_label, compression="gzip") 
    fp.create_dataset("original_label", data=true_label, compression="gzip") 
    fp.close()
    print("embeddings saved")


In [49]:
import h5py


print("training embedding extraction...")
prediction_training, embedding_training, true_label_trainining=embedding_creation(new_train_x, new_train_y)
print("test embedding creation...")
prediction_test, embedding_test,  true_label_test=embedding_creation(new_test_x, new_test_y)
print("new embedding creation...")
prediction_new, embedding_new, true_label_new=embedding_creation(x_new, y_new)
print("degerated data embedding creation...")
#el=False
prediction_deg, embedding_deg, true_label_deg=embedding_creation(x_deg, y_deg)

print("saving the embeddings...")

save_embeddings(embedding_training, prediction_training, true_label_trainining, "VGG-16_II_train.hdf5" )
save_embeddings(embedding_test, prediction_test, true_label_test, "VGG-16_II_test.hdf5" )
save_embeddings(embedding_new, prediction_new, true_label_new, "VGG-16_II_new.hdf5" )
save_embeddings(embedding_deg, prediction_deg, true_label_deg, "VGG-16_II_deg.hdf5" )



training embedding extraction...
188/188 [==============================] - 50s 264ms/step
test embedding creation...
125/125 [==============================] - 34s 269ms/step
new embedding creation...
134/134 [==============================] - 36s 265ms/step
degerated data embedding creation...
87/87 [==============================] - 23s 261ms/step
saving the embeddings...
embeddings saved
embeddings saved
embeddings saved
embeddings saved
